#Feature Extraction

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'gggsdsd:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5281104%2F8784850%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240629%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240629T091424Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da10291b59e02bb18c7e16905adcfe4eeaed2a9fbf98303d9ee9e1f0c376fa1d727084c3c9278235d6c7855a36a243aa50ce69b0b35677cef8e157fef998a1018ec57dd93a2ddbe62df62fc539d810928b9cdd7baed51f72afcd2f1f3c8f18345f4e35314c717abb45c246c9376df0a63bcbde0317615774b92e3ea74113ee6b8edf02f0d65e5db526c7aa6a0f7dd18aa6c0bc7fad1fce71ea3b6d24d385a7db551a92575401a98a4e6f973e5b3b43cb66e996f4a611b76729dbcb7427eee7beffe28e58a534c430fc9b3a92edcbeb5a39349f4bcc329640265906b513b8ed4661dffcf7863e623a06a233abe25f3becb07f3e6d6a3420a84ae6fea39cbbc5488'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gggsdsd/balanced.csv


In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/gggsdsd/balanced.csv',encoding='utf-8')
df.head()  #spoken=1  written=0

,texts,label
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0
2,हमे इन्तजार है ।,1
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1
4,जयपुर,0


In [ ]:
df = df[df['texts'].apply(lambda x: isinstance(x, str))]

In [ ]:
label_counts = df['label'].value_counts()
label_counts

label
0    19344
1    19344
Name: count, dtype: int64

### Features

1.   Number of words in a sentence
2.   Average characters per word in a sentence
3.   Token Type Ratio
4.   Case Marker Percentage
5.   Case Marker Count
6.   Conjunctions Percentage
7.   Conjuctions Count
8.   Hesitation Markers
9.   Wh Questions
10.  Noun Count
11.  Noun Percentage
12.  Adverb Count
13.  Adverb Percentage
14.  Verb Count
15.  Verb Percentage
16.  Personal Pronouns Count
17.  Personal Pronouns Percentage
18.  Depth of Parse Tree

In [ ]:
#Parse Tree Depth
!pip install stanza
import stanza
import numpy as np

# Download and load the Hindi model
stanza.download('hi')
nlp = stanza.Pipeline('hi', processors='tokenize,pos,lemma,depparse')

def get_parse_tree_depth(text):
    doc = nlp(text)
    depths = {}
    for sent in doc.sentences:
        for word in sent.words:
            if word.head == 0:  # Root word
                depths[word.id] = 0
            else:
                depths[word.id] = depths.get(word.head, 0) + 1


    return max(depths.values()) if depths else 0
df['parse_tree_depth'] = df['texts'].apply(get_parse_tree_depth)
df.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 26.4 MB/s eta 0:00:0000:01


,texts,label,parse_tree_depth
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2
2,हमे इन्तजार है ।,1,1
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2
4,जयपुर,0,0


In [ ]:
#Linguistic features
def get_linguistic_features(text):
    # Process the text
    doc = nlp(text)

    features = {
        'noun_count_hi': 0,
        'verb_count_hi': 0,
        'adverb_count_hi': 0,
        'pronoun_count_hi': 0,
        'total_words_hi': 0
    }

    # Count the parts of speech
    for sent in doc.sentences:
        for word in sent.words:
            features['total_words_hi'] += 1
            if word.upos == 'NOUN':
                features['noun_count_hi'] += 1
            elif word.upos == 'VERB':
                features['verb_count_hi'] += 1
            elif word.upos == 'ADV':
                features['adverb_count_hi'] += 1
            elif word.upos == 'PRON':
                features['pronoun_count_hi'] += 1

    # Calculate percentages
    for pos in ['noun', 'verb', 'adverb', 'pronoun']:
        features[f'{pos}_percentage_hi'] = (features[f'{pos}_count_hi'] / features['total_words_hi'] * 100) if features['total_words_hi'] > 0 else 0

    return features
df = df.join(pd.DataFrame(df['texts'].apply(get_linguistic_features).tolist()))
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000


In [ ]:
#Number of words in a sentence
def count_words(sentence):
    return len(sentence.split())
df['words_per_sent'] = df['texts'].apply(count_words)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1


In [ ]:
#Average Characters per word in a sentence
def avg_chars_per_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    total_chars = sum(len(word) for word in words)
    return total_chars / len(words)
df['avg_characters_per_word'] = df['texts'].apply(avg_chars_per_word)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000


In [ ]:
#Token Type Ratio

!pip install indic-nlp-library
from indicnlp.tokenize import indic_tokenize

def token_type_ratio(text):
    tokens = indic_tokenize.trivial_tokenize(text)  # Use Indic NLP tokenizer

    unique_tokens = set(tokens)
    ttr = len(unique_tokens) / len(tokens)

    return ttr

df['token_type_ratio'] = df['texts'].apply(token_type_ratio)
df.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 7.0 MB/s eta 0:00:00


,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407,0.857143
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385,1.000000
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000,1.000000
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909,1.000000
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000,1.000000


In [ ]:
#Case Marker Percentage
import re

def count_case_markers(text):
    case_markers = ["ने", "को", "से", "के", "का", "की", "के", "की", "में", "पर", "तक", "से"]
    words = text.split()


    count = 0
    for word in words:
        if word in case_markers:
            count += 1
    case_marker_percentage = count / len(words) * 100
    return case_marker_percentage

df['case_marker_perc'] = df['texts'].apply(count_case_markers)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407,0.857143,14.814815
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385,1.000000,23.076923
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000,1.000000,0.000000
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909,1.000000,9.090909
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000,1.000000,0.000000


In [ ]:
#Case Marker Count
def count_case_markers(text):
    case_markers = ["ने", "को", "से", "के", "का", "की", "के", "की", "में", "पर", "तक", "से"]
    words = text.split()

    count = 0
    for word in words:
        if word in case_markers:
            count += 1

    return count

df['case_marker_count'] = df['texts'].apply(count_case_markers)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc,case_marker_count
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407,0.857143,14.814815,4
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385,1.000000,23.076923,3
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000,1.000000,0.000000,0
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909,1.000000,9.090909,1
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000,1.000000,0.000000,0


In [ ]:
# Conjunctions in a sentence
conjunctions = [
    "और", "तथा", "पर", "लेकिन", "किन्तु", "मगर", "फिर", "इसलिए", "इसके बाद",
    "इसके पहले", "जबकि", "जैसा कि", "जब तक", "या", "नहीं तो", "इसलिए",
    "जबकी", "इसलिए", "चाहे", "या तो", "कि", "क्योंकि", "जब तक", "जैसे",
    "भले ही", "जितना", "जब", "कि", "चाहे", "कि", "अगर", "चाहे", "वर्तमान में",
    "तब तक", "जबकि", "जो", "यदि", "कि", "किसके बावजूद", "यह बात", "जब तक",
    "जैसा", "जैसे", "जितना", "की", "की", "कि"
]

def count_conjunctions(text):
    words = text.split()
    conjunction_count = 0
    for word in words:
        if word in conjunctions:
            conjunction_count += 1
    return conjunction_count

df['conjunction_count'] = df['texts'].apply(count_conjunctions)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc,case_marker_count,conjunction_count
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407,0.857143,14.814815,4,3
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385,1.000000,23.076923,3,1
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000,1.000000,0.000000,0,0
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909,1.000000,9.090909,1,0
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000,1.000000,0.000000,0,0


In [ ]:
# Percentage of conjunctions in a sentence
def conjunctions_percentage(text):
    words = text.split()
    conjunction_count = 0
    for word in words:
        if word in conjunctions:
            conjunction_count += 1
    conjunction_percentage = (conjunction_count / len(words)) * 100
    return conjunction_percentage

df['conjunctions_percentage'] = df['texts'].apply(conjunctions_percentage)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc,case_marker_count,conjunction_count,conjunctions_percentage
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407,0.857143,14.814815,4,3,11.111111
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385,1.000000,23.076923,3,1,7.692308
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000,1.000000,0.000000,0,0,0.000000
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909,1.000000,9.090909,1,0,0.000000
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000,1.000000,0.000000,0,0,0.000000


In [ ]:
#Hesitation Markers
hesitation_markers = ['अच्छा', 'मतलब', 'वो', 'तो', 'जैसे', 'और', 'क्या', 'है', 'ही', 'ना', 'अब', 'यानी']
def count_hesitation_markers(sentence):
    words = sentence.split()
    return sum(word in hesitation_markers for word in words)
df['hesitation_markers'] = df['texts'].apply(count_hesitation_markers)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,adverb_percentage_hi,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc,case_marker_count,conjunction_count,conjunctions_percentage,hesitation_markers
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,0.0,0.000000,27,4.407407,0.857143,14.814815,4,3,11.111111,2
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,0.0,7.692308,13,4.615385,1.000000,23.076923,3,1,7.692308,0
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,0.0,25.000000,4,3.250000,1.000000,0.000000,0,0,0.000000,1
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,0.0,9.090909,11,3.090909,1.000000,9.090909,1,0,0.000000,1
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,0.0,0.000000,1,5.000000,1.000000,0.000000,0,0,0.000000,0


In [ ]:
# Wh-questions in Hindi
wh_questions = ["कौन", "क्या", "कब", "कहाँ", "कैसे", "क्यों"]
def count_wh_questions(text):
    words = text.split()
    wh_question_count = 0
    for word in words:
        if word in wh_questions:
            wh_question_count += 1
    return wh_question_count

df['wh_question_count'] = df['texts'].apply(count_wh_questions)
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,...,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc,case_marker_count,conjunction_count,conjunctions_percentage,hesitation_markers,wh_question_count
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,...,0.000000,27,4.407407,0.857143,14.814815,4,3,11.111111,2,0
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,...,7.692308,13,4.615385,1.000000,23.076923,3,1,7.692308,0,0
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,...,25.000000,4,3.250000,1.000000,0.000000,0,0,0.000000,1,0
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,...,9.090909,11,3.090909,1.000000,9.090909,1,0,0.000000,1,0
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,...,0.000000,1,5.000000,1.000000,0.000000,0,0,0.000000,0,0


In [ ]:
df.to_csv('my_dataframe.csv', encoding = 'utf-8',index=False)

In [ ]:
df.head()

,texts,label,parse_tree_depth,noun_count_hi,verb_count_hi,adverb_count_hi,pronoun_count_hi,total_words_hi,noun_percentage_hi,verb_percentage_hi,...,pronoun_percentage_hi,words_per_sent,avg_characters_per_word,token_type_ratio,case_marker_perc,case_marker_count,conjunction_count,conjunctions_percentage,hesitation_markers,wh_question_count
0,अमेरिकी और रूसी पुलिस के संयुक्त अभियान के फलस...,0,2,7,2,0,0,28,25.000000,7.142857,...,0.000000,27,4.407407,0.857143,14.814815,4,3,11.111111,2,0
1,आज यहां रांची एक्सप्रेस के साथ बातचीत में वित्...,0,2,2,1,0,1,13,15.384615,7.692308,...,7.692308,13,4.615385,1.000000,23.076923,3,1,7.692308,0,0
2,हमे इन्तजार है ।,1,1,1,0,0,1,4,25.000000,0.000000,...,25.000000,4,3.250000,1.000000,0.000000,0,0,0.000000,1,0
3,जिस के नतीजे मे ये हुक्म सादिर किया गया है ।,1,2,3,1,0,1,11,27.272727,9.090909,...,9.090909,11,3.090909,1.000000,9.090909,1,0,0.000000,1,0
4,जयपुर,0,0,0,0,0,0,1,0.000000,0.000000,...,0.000000,1,5.000000,1.000000,0.000000,0,0,0.000000,0,0
